Kosmos 2.5
1. discard figure and put caption to the end
2. put table at begining of page and convert to html (different to the kosmos-2.5 paper?)

In [1]:
import torch
import pymupdf
from PIL import Image
from transformers import AutoProcessor, Kosmos2_5ForConditionalGeneration

repo = "microsoft/kosmos-2.5"
device = "cuda:7"
dtype = torch.bfloat16
model = Kosmos2_5ForConditionalGeneration.from_pretrained(repo, device_map=device, torch_dtype=dtype)
processor = AutoProcessor.from_pretrained(repo)

/home/duan/miniconda3/envs/copylookup/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Kosmos2_5TextForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an excepti

The OrderedVocab you are attempting to save contains holes for indices [100260, 100265, 100266, 100267, 100268, 100269, 100270, 100271, 100272, 100273, 100274, 100275, 100276, 100277, 100278, 100279, 100281], your vocabulary could be corrupted !


In [4]:
# sample image
pixmap = pymupdf.open("./2307.00421.pdf")[5].get_pixmap(dpi=800)
mode = 'RGB'
img = Image.frombytes(mode, [pixmap.width, pixmap.height], pixmap.samples)

prompt = "<md>"
inputs = processor(text=prompt, images=img, return_tensors="pt")

height, width = inputs.pop("height"), inputs.pop("width")
raw_width, raw_height = img.size
scale_height = raw_height / height
scale_width = raw_width / width

inputs = {k: v.to(device) if v is not None else None for k, v in inputs.items()}
inputs["flattened_patches"] = inputs["flattened_patches"].to(dtype)
generated_ids = model.generate(
    **inputs,
    max_new_tokens=1024,
)

generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)
print(generated_text[0])

<table>
<thead>
<tr>
<th>
Brightness range
</th>
<th>
Original
</th>
<th>
Color transfer
</th>
<th>
Gaussian blur
</th>
</tr>
</thead>
<tbody>
<tr>
<td>
1 (AdvPatch)
</td>
<td>
89.4%
</td>
<td>
90.8%
</td>
<td>
47.8%
</td>
</tr>
<tr>
<td>
0.35
</td>
<td>
89.5%
</td>
<td>
87.9%
</td>
<td>
22.7%
</td>
</tr>
<tr>
<td>
0.24
</td>
<td>
74.2%
</td>
<td>
75.3%
</td>
<td>
10.1%
</td>
</tr>
</tbody>
</table>

Table 1: Performance with color transfer and Gaussian blur

<table>
<thead>
<tr>
<th>
Brightness range
</th>
<th>
Original
</th>
<th>
10% drift
</th>
<th>
15% drift
</th>
<th>
20% drift
</th>
</tr>
</thead>
<tbody>
<tr>
<td>
1 (AdvPatch)
</td>
<td>
89.4%
</td>
<td>
87.6%
</td>
<td>
85.9%
</td>
<td>
83.3%
</td>
</tr>
<tr>
<td>
0.35
</td>
<td>
89.5%
</td>
<td>
84.2%
</td>
<td>
77.6%
</td>
<td>
67.2%
</td>
</tr>
<tr>
<td>
0.24
</td>
<td>
74.2%
</td>
<td>
68.0%
</td>
<td>
43.2%
</td>
<td>
27.3%
</td>
</tr>
</tbody>
</table>

Table 2: Performance with different color drift

### Effectiveness of